## 批量统计词频（多线程 vs 常规）
* 词频统计(按小时，i.e., 按集合统计)
* 处理24小时的词频，提取每小时top200的高频词，并保存在文件中
* 存疑：感觉多线程也没有很快，有待分析...

In [1]:
import pymongo
con_str = "localhost:27017"
db = pymongo.MongoClient(con_str, serverSelectionTimeoutMS = 10000)
try:
    # print(db.server_info())
    
    # 查询数据库中所有数据库的名称
    db_names = db.list_database_names()
    print(db_names)
except Exception:
    print("connect failed!")
    
targetDB = 'db_Weibo_SH_0331_All'

if targetDB in db_names:
    db_Weibo_SH_0331_All = db['db_Weibo_SH_0331_All']
    print(targetDB + ": 已连接！")

['admin', 'config', 'db_Weibo_SH_0331_All', 'db_tvcgR3', 'db_weiboDB', 'local', 'test', 'weiboInfoAll']
db_Weibo_SH_0331_All: 已连接！


In [134]:
# key：集合的名称
# value: 集合中所有词的数量，不一样的词的数量，词频字典

# wordFreqDict = {
#     'col_weibo_timePeriod0' : [17104459, 63090, {'抗疫': 77797, '11': 75271, '12345': 74553, ...}]
#     'col_weibo_timePeriod1' : [14104454, 6308, {'疫情': 77797, '11': 75271, '12345': 74553, ...}]
# }

def computeWordFreq(targetCol, wordFreqDict, topK):
    
    wordFreqDict[targetCol] = list()
    
    exec(targetCol + " = db_Weibo_SH_0331_All['" + targetCol+ "']")
    exec("recs = " + targetCol + ".find()")
    loc = locals()
    
    tempCnt_Words = 0  # ●1 存放当前集合中的词的总数

    freqWords = {}  # key(词汇)，value(词频数)

    for cW in loc['recs']:
        # 手动写词频统计函数，基于字典实现
        currentContent = cW.get('plusCutting')
        tempCnt_Words = tempCnt_Words + len(currentContent)  # 存放当前集合中的词的总数

        for eachWord in currentContent:
            freqWords[eachWord] = freqWords.get(eachWord, 0) + 1

    items = list(freqWords.items())  # 转换为list
    # print("items.len = " + str(len(items)))  # ●2 不一样的词的个数
    items.sort(key=lambda x: x[1], reverse=True)  # 按照词频排序

    wordcloudDic = dict()  # ●3 存放前k个词 & 词频
    for i in range(topK):
        word, count = items[i]
        wordcloudDic[word] = count
        # print("{0:<20}{1:>10}".format(word, count))
        
    wordFreqDict[targetCol].append(tempCnt_Words)
    wordFreqDict[targetCol].append(len(items))
    wordFreqDict[targetCol].append(wordcloudDic)



In [137]:
# 没用多线程
import time

if __name__ == '__main__':
    startTime = time.time() 
    
    # 循环操作24个集合：新建指定字段
    subCol24 = list()
    subCol24.append(["col_weibo_timePeriod" + str(period) for period in range(0, 24)])
    print(subCol24[0])
    colPackages = subCol24[0]

    wordFreqDictNoThread = dict()
    
    topK = 200  # 返回的是前200个高频词
    
#     threads = []
    for i in range(0, len(colPackages)):
        print(colPackages[i])
        computeWordFreq(colPackages[i], wordFreqDictNoThread, topK)
        
#         t = myThread("col" + str(i), colPackages[i], wordFreqDict, topK)
#         t.start()
#         threads.append(t)
        
#     [thread.join() for thread in threads]
#     print ("退出主线程")
    
    endTime = time.time()    
    print("---------------> Time consuming: %.2f" % (endTime - startTime))
    save_variable1(wordFreqDictNoThread, 'wordFreqDictNoThread_WithType.txt')
    save_variable2(str(wordFreqDictNoThread), 'wordFreqDictNoThread_WithoutType.txt')  # 只能查看字符串

['col_weibo_timePeriod0', 'col_weibo_timePeriod1', 'col_weibo_timePeriod2', 'col_weibo_timePeriod3', 'col_weibo_timePeriod4', 'col_weibo_timePeriod5', 'col_weibo_timePeriod6', 'col_weibo_timePeriod7', 'col_weibo_timePeriod8', 'col_weibo_timePeriod9', 'col_weibo_timePeriod10', 'col_weibo_timePeriod11', 'col_weibo_timePeriod12', 'col_weibo_timePeriod13', 'col_weibo_timePeriod14', 'col_weibo_timePeriod15', 'col_weibo_timePeriod16', 'col_weibo_timePeriod17', 'col_weibo_timePeriod18', 'col_weibo_timePeriod19', 'col_weibo_timePeriod20', 'col_weibo_timePeriod21', 'col_weibo_timePeriod22', 'col_weibo_timePeriod23']
col_weibo_timePeriod0
col_weibo_timePeriod1
col_weibo_timePeriod2
col_weibo_timePeriod3
col_weibo_timePeriod4
col_weibo_timePeriod5
col_weibo_timePeriod6
col_weibo_timePeriod7
col_weibo_timePeriod8
col_weibo_timePeriod9
col_weibo_timePeriod10
col_weibo_timePeriod11
col_weibo_timePeriod12
col_weibo_timePeriod13
col_weibo_timePeriod14
col_weibo_timePeriod15
col_weibo_timePeriod16
col_

In [135]:
import threading
import time

class myThread(threading.Thread):
    def __init__(self, threadName, targetCol, wordFreqDict, topK):
        threading.Thread.__init__(self)
        self.threadName = threadName
        self.targetCol = targetCol
        self.wordFreqDict = wordFreqDict
        self.topK = topK
        
    def run(self):
        print ("开始线程：" + str(self.threadName))
        computeWordFreq(self.targetCol, self.wordFreqDict, self.topK)
        print ("退出线程：" + str(self.threadName))

In [139]:
# 用了多线程
import time

if __name__ == '__main__':
    startTime = time.time() 
    
    # 循环操作24个集合：新建指定字段
    subCol24 = list()
    subCol24.append(["col_weibo_timePeriod" + str(period) for period in range(0, 24)])
    print(subCol24[0])
    colPackages = subCol24[0]

    wordFreqDictMulThread = dict()
    
    topK = 200  # 返回的是前200个高频词
    
    threads = []
    for i in range(0, len(colPackages)):
        print(colPackages[i])
        t = myThread("col" + str(i), colPackages[i], wordFreqDictMulThread, topK)
        t.start()
        threads.append(t)
        
    [thread.join() for thread in threads]
    print ("退出主线程")
    
    endTime = time.time()    
    print("---------------> Time consuming: %.2f" % (endTime - startTime))
    save_variable1(wordFreqDictMulThread, 'wordFreqDictMulThread_WithType.txt')
    save_variable2(str(wordFreqDictMulThread), 'wordFreqDictMulThread_WithoutType.txt')  # 只能查看字符串

['col_weibo_timePeriod0', 'col_weibo_timePeriod1', 'col_weibo_timePeriod2', 'col_weibo_timePeriod3', 'col_weibo_timePeriod4', 'col_weibo_timePeriod5', 'col_weibo_timePeriod6', 'col_weibo_timePeriod7', 'col_weibo_timePeriod8', 'col_weibo_timePeriod9', 'col_weibo_timePeriod10', 'col_weibo_timePeriod11', 'col_weibo_timePeriod12', 'col_weibo_timePeriod13', 'col_weibo_timePeriod14', 'col_weibo_timePeriod15', 'col_weibo_timePeriod16', 'col_weibo_timePeriod17', 'col_weibo_timePeriod18', 'col_weibo_timePeriod19', 'col_weibo_timePeriod20', 'col_weibo_timePeriod21', 'col_weibo_timePeriod22', 'col_weibo_timePeriod23']
col_weibo_timePeriod0
开始线程：col0
col_weibo_timePeriod1
开始线程：col1
col_weibo_timePeriod2
开始线程：col2col_weibo_timePeriod3

开始线程：col3
col_weibo_timePeriod4
开始线程：col4
col_weibo_timePeriod5
开始线程：col5col_weibo_timePeriod6

开始线程：col6
col_weibo_timePeriod7
开始线程：col7
col_weibo_timePeriod8
开始线程：col8
col_weibo_timePeriod9
开始线程：col9col_weibo_timePeriod10

开始线程：col10
col_weibo_timePeriod11
开始线程：col